In [1]:
from pyspark import SparkContext, SparkConf
from sagemaker_pyspark import IAMRole
from sagemaker_pyspark.algorithms import KMeansSageMakerEstimator
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.tree import DecisionTree
from pyspark.mllib.util import MLUtils
from pyspark.mllib.tree import RandomForest
import time
import numpy as np

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1580131131036_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

we add some env variables so we can use S3 

In [2]:
iam_role = "arn:aws:iam::114820912377:role/service-role/AmazonSageMaker-ExecutionRole-20200112T113996"
region = "eu-west-3"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
from pyspark.mllib.linalg import Vectors

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:


# Load and parse the data file into an RDD of LabeledPoint.
data = MLUtils.loadLibSVMFile(sc, 's3://ot6spark/libsvm_data/covtype.data.libsvm_v3')
#testData =  MLUtils.loadLibSVMFile(sc, 's3://ot6spark/libsvm_data/news20.t')

# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.9, 0.1])


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
iterations = 1

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
times = []

for i in range(iterations):
    print(str(i))
    start_time = time.time()
    model = RandomForest.trainClassifier(trainingData, numTrees=15, numClasses = 8,featureSubsetStrategy="auto", categoricalFeaturesInfo={},impurity='gini', maxDepth=30, maxBins=32)
    end_time = time.time()
    times.append(end_time - start_time)
    
print('the train time is = '+str(np.mean(times))+'the max is = '+str(max(times))+'the min is = '+str(min(times))+' and the std is = '+str(np.std(times)))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0
the train time is = 134.5738399028778the max is = 134.573839903the min is = 134.573839903 and the std is = 0.0

In [8]:

times = []
for i in range(iterations):
    start_time = time.time()
    predictions = model.predict(testData.map(lambda x: x.features))
    end_time = time.time()
    times.append(end_time - start_time)
    
print('the test time is = '+str(np.mean(times))+'the max is = '+str(max(times))+'the min is = '+str(min(times))+' and the std is = '+str(np.std(times)))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

the test time is = 1.3863871097564697the max is = 1.38638710976the min is = 1.38638710976 and the std is = 0.0

## calculate the accuracy 

In [67]:
def get_type(class_numbre):
    classes = ["Spruce-Fir","Lodgepole Pine","Ponderosa Pine","Cottonwood/Willow","Aspen","Douglas-fir","Krummholz"]
    return classes[class_numbre -1]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)
testErr = labelsAndPredictions.filter(lambda (v, p): v != p).count() / float(testData.count())
accuracy  = labelsAndPredictions.filter(lambda (v, p): v == p).count() / float(testData.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
print('Test Error = ' + str(testErr))
print('Accuracy is = '+str(accuracy))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Test Error = 0.136904659375
Accuracy is = 0.863095340625

example get some data from test data and send back the result 

In [43]:
(a, _) = testData.randomSplit([0.000005, 0.1])
a.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

4

In [46]:
pre = model.predict(a.map(lambda x: x.features))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [71]:
print("the true classes =",str([get_type(int(i)) for i in pre.take(4)]))
print("the predicted classes =",str([get_type(int(i.label)) for i in a.take(4)]))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

('the true classes =', "['Douglas-fir', 'Spruce-Fir', 'Krummholz', 'Spruce-Fir']")
('the predicted classes =', "['Ponderosa Pine', 'Spruce-Fir', 'Krummholz', 'Spruce-Fir']")

In [65]:
a.take(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[LabeledPoint(3.0, (54,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53],[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1195.0,106.0,186.0,207.0,1082.0,58.0,150.0,22.0,30.0,2179.0]))]